In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
data=pd.read_csv('/content/drive/My Drive/abcd/Reviews.csv')
col=['Summary','Text']
data[col].to_csv('Modified_review.csv',index=False)

In [ ]:
del data

In [ ]:
df=pd.read_csv('/content/drive/My Drive/abcd/Modified_review.csv')
rows=df[df['Summary'].isna() | df['Text'].isna()].shape[0]
print(rows)

27


In [ ]:
df.dropna(subset=['Summary','Text'],inplace=True)

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import string

In [ ]:
def html_remove(text):
    word=BeautifulSoup(text,"html.parser")
    return word.get_text()

In [ ]:
def accented(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [ ]:
def preprocess(text):
    text = html_remove(text)
    text = accented(text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.lower()
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [word for word in tokens if word not in string.punctuation]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return ' '.join(lemmatized_tokens)

df['Summary'] = df['Summary'].apply(preprocess)
df['Text'] = df['Text'].apply(preprocess)

df.to_csv('Preprocessed_Review1.csv', index=False)


<ipython-input-8-262890ab649b>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  word=BeautifulSoup(text,"html.parser")
<ipython-input-8-262890ab649b>:2: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  word=BeautifulSoup(text,"html.parser")
<ipython-input-8-262890ab649b>:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  word=BeautifulSoup(text,"html.parser")


In [ ]:
df = pd.read_csv('Preprocessed_Review1.csv')
sampled_df = df.sample(frac=0.30, random_state=42)

sampled_df.to_csv('/content/drive/My Drive/abcd/less_data.csv', index=False)


In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(sampled_df, test_size=0.25, random_state=42)

train_df.to_csv('/content/drive/My Drive/abcd/train.csv', index=False)
test_df.to_csv('/content/drive/My Drive/abcd/test.csv', index=False)

In [ ]:
n1=train_df.shape[0]
n2=test_df.shape[0]
print(n1)
print(n2)

127896
42632


In [ ]:
!pip install transformers

In [ ]:
pip install transformers[torch]

In [ ]:
!pip install accelerate -U


In [ ]:
!pip show accelerate


Name: accelerate
Version: 0.29.3
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 


In [ ]:
import pandas as pd
import re
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer


In [ ]:
model_path = '/content/drive/My Drive/abcd/train_output'
model = load_model(model_path)
tokenizer = load_tokenizer(model_path)


In [ ]:
import pandas as pd
import random
texts=[]
summaries=[]
df1=pd.read_csv("/content/drive/My Drive/abcd/train.csv")

num_rows = len(df1)
random_indices = random.sample(range(num_rows), 10)
index_counter = 0
while len(texts) < 10:
    random_index = random_indices[index_counter]
    text = df1.iloc[random_index]['Text']
    summary = df1.iloc[random_index]['Summary']
    texts.append(text)
    summaries.append(summary)
    index_counter += 1

for text, summary in zip(texts, summaries):
    print("Text:", text)
    print("Summary:", summary)
    print()




Text: highly recommend rescue remedy purchased product friend dog 3yr old 85 lb male yellow lab named jack jack always frightened thunderstorm would pant drool shed fur tremble friend applied 4 drop treat gave jack within moment mood settled calm peaceful theyve two thunderstorm since giving jack rescue remedy time calm seems unbothered weather jack bit groggy next day side affect noticed
Summary: work like charm

Text: greenies 96 received quickly protective box fresh resealable bag would order vendor
Summary: greenies dental chew

Text: use litter automatic litter box work wonder clump hard clear away clean leaf odor behind bit dusty pour bunch dont find worse others ive tried occasion tried another brand litter always severely disappointed highly recommend litter especially automatic litter box
Summary: strong clumping great smell

Text: exactly ordered super fast delivery much easier grocery shopping expecially common ingredient
Summary: great

Text: perfect new puppy love smell re

In [ ]:
!pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 25.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.2
    Uninstalling transformers-4.38.2:
      Successfully uninstalled transformers-4.38.2


In [ ]:
from transformers import GPT2Tokenizer
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2LMHeadModel, AdamW
import torch

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

class Modified_Set(Dataset):
    def __init__(self, texts, summaries, tokenizer, max_length=512):
        self.texts = texts
        self.summaries = summaries
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        summary = self.summaries[idx]
        combined_text = (text, '[SEP]', summary)
        encoding = self.tokenizer(combined_text, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="pt")
        input_ids = encoding.input_ids.squeeze()
        attention_mask = encoding.attention_mask.squeeze()
        labels = input_ids.clone()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }


modified_data= Modified_Set(texts, summaries, tokenizer)

batch_size = 2
dataloader = DataLoader(modified_data, batch_size=batch_size, shuffle=True)
model = GPT2LMHeadModel.from_pretrained('gpt2')
optimizer = AdamW(model.parameters(), lr=5e-5)
epochs = 3
model.train()

def forward_pass(model, input_ids, attention_mask, labels):
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    return loss

def backward_pass(loss, optimizer):
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

def train_model(model, dataloader, optimizer, epochs):
    for epoch in range(epochs):
        total_loss = 0
        for batch in dataloader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
            loss = forward_pass(model, input_ids, attention_mask, labels)
            backward_pass(loss, optimizer)
            total_loss += loss.item()
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}: Average Loss: {avg_loss}")

train_model(model, dataloader, optimizer, epochs=3)
model.save_pretrained('fine_tuned_gpt2_e2')


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import pandas as pd
test_texts=[]
test_summaries=[]
test_df=df1.head(10)

for i, row in test_df.iterrows():
    test_texts.append(row['Text'])
    test_summaries.append(row['Summary'])

In [ ]:
!pip install rouge

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from rouge import Rouge
import pandas as pd

def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model

def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    return tokenizer

def generate_summary(model, tokenizer, sequence, max_length):
    inputs = tokenizer.encode(sequence, return_tensors='pt', max_length=max_length, truncation=True)
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1, early_stopping=True)
    generated_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_summary

def calculate_rouge(generated_summaries, given_summaries):
    rouge = Rouge()
    scores = rouge.get_scores(generated_summaries, given_summaries, avg=True)
    return scores

model_path = "fine_tuned_gpt2"
model = load_model(model_path)
tokenizer = load_tokenizer(model_path)

generated_summaries = []
test_summaries = []
generated_summaries

def extract_review_sentiment(summary):
    remove=summary.find("None")
    remove_word="None"
    if remove!=-1:
      new_summary=summary[remove + len(remove_word):].strip()
    else:
      new_summary="None"
    return new_summary
for text in test_texts:
    generated_summary = generate_summary(model, tokenizer, text, max_length=500)
    generated_summary = extract_review_sentiment(generated_summary)  # Adjust max_length as needed
    generated_summaries.append(generated_summary)

test_summaries = test_df['Summary'].tolist()

rouge_scores = calculate_rouge(generated_summaries, test_summaries)

print("ROUGE Scores:")
print(rouge_scores)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

ROUGE Scores:
{'rouge-1': {'r': 0.4133333333333333, 'p': 0.029705520524115948, 'f': 0.05460852674803925}, 'rouge-2': {'r': 0.175, 'p': 0.008955051014456955, 'f': 0.016831678857494748}, 'rouge-l': {'r': 0.3933333333333333, 'p': 0.02692774274633817, 'f': 0.04973047796755146}}


In [ ]:
generated_summaries

['tried carroll shelbys chili kit earlier week excellent tried one tonight using cut meat disappointed one peppery enough seemed lack cumin garlic firey bland thin tasting rich',
 'upming authentic mexican style salsa mpany formerly tucson arizona located bend oregon wanted make gift basket customer needed product fill garden eatin blue tortilla chip fit theme hit mark ntinue product additional gift basket well need business grows southwestsalsam',
 'generally love haribo product one didnt like flavor texture son however thought okay',
 'youre looking bargain basement cholate isnt want try taste youll never forget probably buy cant go wrong simply best ive ever bar none expensive so acura worth',
 'favorite uncle diabetic always feel awful invite birthday cant join eating cake rest u picked package used okie crust cream pie jello sugarfree instant pudding pie filling cholate ounce box pack whipped cream lightly sweetened nonsugar sweetener result gorgeous tasted pretty good even u suga